# A biblioteca Plotly<a id='home'></a> 

Arquivos necessários: Nenhum

A visualização de dados eficaz requer que você conte uma história, exponha seu ponto de vista ou talvez convença seu colega, supervisor ou CEO sobre algo interessante nos dados. Até agora estamos usando matplotlib para gráficos porque o pacote é:

1. Rápido
2. Fácil de usar 
3. Brinca bem com os pandas

Também é chato - um ponto com o qual até os criadores do matplotlib concordam

> _Matplotlib é implementado principalmente para plotagem básica._ — Documentação do Matplotlib

Claro, seaborn adiciona alguma funcionalidade e torna as figuras mais fortes, mas no final do dia, essas são todas "plotagens estáticas"; ou seja, eles só podem dizer o que é mostrado na tela para que o leitor não possa mergulhar mais fundo nos plots passando o mouse sobre os pontos para descobrir informações ou adicionar filtros para modificar os gráficos ao vivo.

Todos nós plotamos que são capazes de realizar funções como essas ***Visualizações interativas***. As visualizações interativas são ótimas porque podemos incluir uma tonelada de informações nos gráficos sem que a figura fique confusa. Além disso, eles parecem muuuuito mais legais.

O objetivo desta palestra é baixar e manipular dados usando uma variedade de fontes de *application programming interface* (api) para criar figuras úteis que documentem o desempenho econômico ao longo do tempo usando o pacote __Plotly__ figure. Plotly será a chave para o desenvolvimento de visualizações interativas, como

* Todos os gráficos Matplotlib e Seaborn.
* Gráficos Estatísticos que incluem, mas não se limitam a Categorias Paralelas e Gráficos de Árvore de Probabilidades.
* Gráficos científicos que você nunca pensou, desde gráficos de rede até gráficos de radar.
* Gráficos financeiros que são úteis para análise de séries temporais, exemplos incluem castiçais, funis e gráficos de marcadores.
* Mapas geológicos e parcelas tridimensionais que permitem interagir com eles.

Além disso, o Plotly é apoiado por uma empresa que faz visualizações interativas baseadas na web e aplicativos da web com o mesmo nome. Eu não vou cobrir isso aqui, mas você pode enviar suas figuras plotadas para a nave-mãe para terminar através de seu aplicativo baseado na web.

Instale plotly como qualquer outro pacote; ou seja,

`pip install plotly`

O pacote plotly Python existe para criar, manipular e renderizar figuras gráficas (ou seja, gráficos, plotagens, mapas e diagramas) representadas por estruturas de dados também chamadas de figuras. O processo de renderização usa a biblioteca JavaScript Plotly.js nos bastidores, embora os desenvolvedores de Python que usam esse módulo raramente precisem interagir diretamente com a biblioteca Javascript -- se é que o fazem. As figuras podem ser representadas em Python como `dicts` ou como instâncias da classe `plotly.graph_objects.Figure` e são serializadas como texto em JavaScript Object Notation (JSON) antes de serem passadas para Plotly.js. Tecnicamente, isso significa que podemos gerar uma figura apenas definindo um `dict` e usando a função de baixo nível `plotly.io.show`.


<a id = "agenda"> </a>
## Agenda

1. [Carregar dados usando APIs](#load_data)
2. [Gerar Números de Desempenho Econômico-Financeiro](#figuras)
3. [Termos presidenciais sobrepostos no cargo](#presidentes)
4. [Combinar figuras para produzir código HTML que pode ser incorporado em uma página da web](#html)

In [1]:
import plotly.io as pio
fig = dict({
    "data": [{"type": "bar",
              "x": [1, 2, 3],
              "y": [1, 3, 2]}],
    "layout": {"title": {"text": "Sample Figure Specified via Python Dictionary and Matplotlib"}}
})

pio.show(fig)

Observe que não fizemos nada extravagante, mas essa figura é realmente bem diferente de qualquer coisa que criamos antes - é interativa. Para ver isso, passe o cursor sobre as barras.

Na verdade, usaremos um módulo `plotly.express` mais simples e de nível superior (leia-se: amigável ao usuário), também conhecido como "Plotly Express". Este módulo é bom porque consiste em funções Python que retornam objetos `plotly.graph_objects.Figure` totalmente preenchidos, o que torna nossa vida mais fácil.

Os objetos Graph têm vários benefícios em comparação com os dicionários Python simples:

1. Objetos de gráfico gritarão com você se você usar um nome de propriedade inválido ou um valor de propriedade inválido como chave para um objeto de gráfico. O computador (provavelmente) fornecerá uma mensagem de erro útil descrevendo o problema. Os dicionários Python simplesmente ignorarão seu erro.


2. Objetos de gráfico contêm descrições de cada propriedade válida que você pode usar para aprender sobre as propriedades disponíveis como uma alternativa para consultar a Referência Completa online.


3. As propriedades dos objetos de gráfico podem ser acessadas usando a pesquisa de chave no estilo de dicionário (por exemplo, fig["layout"]) ou acesso à propriedade no estilo de classe (por exemplo, fig.layout).


4. Os objetos Graph suportam funções de conveniência de nível superior para fazer atualizações em figuras já construídas (por exemplo, .update_layout(), .add_trace()). Usaremos ambos extensivamente abaixo.


5. Construtores de objetos gráficos e métodos de atualização aceitam "sublinhados mágicos" (por exemplo, go.Figure(layout_title_text="The Title") em vez de dict(layout=dict(title=dict(text="The Title")))) para mais código compacto (e mais fácil de ler). Mais uma vez, vamos aproveitar isso.


6. Objetos de gráfico suportam renderização anexada (.show()) e funções de exportação (.write_image()) que invocam automaticamente as funções apropriadas do módulo plotly.io.

Em resumo, usaremos `plotly.express` em vez de `plotly.io` porque é mais amigável. Primeiro, vamos carregar alguns dados.

<a id="load_data"></a>
# 1. Carregar dados via APIs ([top](#agenda))

In [2]:
# Load necessary packages
import pandas as pd
import plotly.express as px        # high-level plotly module
import plotly.graph_objects as go  # lower-level plotly module with more functions
import pandas_datareader as pdr    # we are grabbing the data and wb functions from the package
import datetime as dt              # for time and date
import requests # api module

In [3]:
!pip install plotly

## Desempenho do mercado de ações via Yahoo! API de finanças

Aqui iremos utilizar a API da yahoo finance para recuperar dados de índices financeiros, tais como Nasdaq, Dow Jones, S&P, Bitcoin etc.

In [4]:
!pip install yfinance

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 10.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
import yfinance as yf

In [6]:
yf.download?

In [7]:
# Definimos o range para aquisição dos dados
start = dt.datetime(2015, 1, 1)
end = dt.date.today()

stocks = yf.download("^DJI", start, end)
btc = yf.download("BTC-USD", start, end)

stocks.reset_index(inplace=True)
btc.reset_index(inplace=True)



stocks.columns = [col.lower() for col in stocks.columns]
stocks['date'] = pd.to_datetime(stocks['date'], yearfirst=True)
stocks.set_index('date', inplace = True)

btc.columns = [col.lower() for col in btc.columns]
btc['date'] = pd.to_datetime(btc['date'], yearfirst=True)
btc.set_index('date', inplace = True)

stocks['date'] = [dt.datetime.strftime(i, "%Y-%m-%d") for i in stocks.index]
stocks['date'] = pd.to_datetime(stocks['date'], yearfirst=True)
stocks.set_index('date', inplace = True)

btc['date'] = [dt.datetime.strftime(i, "%Y-%m-%d") for i in btc.index]
btc['date'] = pd.to_datetime(btc['date'], yearfirst=True)
btc.set_index('date', inplace = True)

# Change frequency to quarterly level
#stocks = stocks.resample('q').mean()

# Generate quarterly returns
#stocks['return'] = stocks['close'].pct_change()*100


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
stocks.head()

,open,high,low,close,adj close,volume
date,,,,,,
2015-01-02,17823.070312,17951.779297,17731.300781,17832.990234,17832.990234,76270000
2015-01-05,17821.300781,17821.300781,17475.929688,17501.650391,17501.650391,116160000
2015-01-06,17504.179688,17581.050781,17262.369141,17371.640625,17371.640625,101870000
2015-01-07,17374.779297,17597.080078,17374.779297,17584.519531,17584.519531,91030000
2015-01-08,17591.970703,17916.039062,17591.970703,17907.869141,17907.869141,114890000


In [9]:
btc.head()

,open,high,low,close,adj close,volume
date,,,,,,
2015-01-01,320.434998,320.434998,314.002991,314.248993,314.248993,8036550
2015-01-02,314.079010,315.838989,313.565002,315.032013,315.032013,7860650
2015-01-03,314.846008,315.149994,281.082001,281.082001,281.082001,33054400
2015-01-04,281.145996,287.230011,257.612000,264.195007,264.195007,55629100
2015-01-05,265.084015,278.341003,265.084015,274.473999,274.473999,43962800


## <font color='orange'>Figura 1:</font> Desempenho do mercado de ações e Bitcoin

Vamos dar uma olhada no desempenho de fechamento do Bitcoin comparado ao índice Dow Jones

In [10]:
figPx = go.Figure()
figPx.add_trace(go.Scatter(x=stocks.index, y=stocks["close"], name='Dow Jones',
                         line=dict(color='blue', width=1)))
figPx.add_trace(go.Scatter(x=btc.index, y=btc["close"], name='Bitcoin',
                         line=dict(color='firebrick', width=1)))

In [23]:
figPx = go.Figure()
figPx.add_trace(go.Scatter(x=stocks.index, y=stocks["close"], name='Dow Jones',
                         line=dict(color='blue', width=1)))
figPx.add_trace(go.Scatter(x=btc.index, y=btc["close"], name='Bitcoin',
                         line=dict(color='firebrick', width=1)))
figPx.update_traces(hovertemplate='Value: %{y:.1f}')
figPx.update_layout(hovermode="x unified",
                        plot_bgcolor='rgba(0,0,0,0)',
                        title={'x':0.5,'xanchor': 'center',
                              'font':{'size': 24}
                        },
                        xaxis=dict(
                            title="Year",
                            showgrid=False,  # Removes X-axis grid lines
                            dtick= 'M48'
                        ),
                        yaxis=dict(
                            title="Index Value",  
                            tickformat = ',f',  # Add thousands
                            showgrid=False,  # Removes Y-axis grid lines    
                            zeroline=False
                        ))

# annotate the figure with Covid info
figPx.add_annotation(
            x=dt.datetime(2020, 3, 1),
            y=25000,
            xref="x",
            yref="y",
            text="<b>Covid-19 Declared</b><br><b> a Pandemic</b><br><b> (March 2020)</b>",
            showarrow=True,
            arrowhead=5,
            ax=80,
            ay=-160)

figPx.update_xaxes(showline=True, linewidth=2, linecolor='black')
figPx.update_yaxes(showline=True, linewidth=2, linecolor='black')

Talvez seja mais interessante exibir essas informações como gráfico candlestick, vamos fazer:

In [16]:
figPx = go.Figure()
figPx.add_trace(go.Candlestick(x=stocks.index, 
                               open=stocks["open"], 
                               high=stocks["high"], 
                               low=stocks["low"], 
                               close=stocks["close"], 
                               name='Dow Jones',
                               increasing_line_color= 'green', decreasing_line_color= 'orange'
                         ))
figPx.add_trace(go.Candlestick(x=btc.index, 
                               open=btc["open"], 
                               high=btc["high"], 
                               low=btc["low"], 
                               close=btc["close"],
                               name='Bitcoin',
                               increasing_line_color= 'blue', decreasing_line_color= 'red'
                         ))
figPx.update_xaxes(showline=True, linewidth=2, linecolor='black',range=['2022-03-01','2022-04-01'])
figPx.update_yaxes(showline=True, linewidth=2, linecolor='black',range=[30000,50000])

In [26]:
figPx = go.Figure()
figPx.add_trace(go.Candlestick(x=stocks.index, 
                               open=stocks["open"], 
                               high=stocks["high"], 
                               low=stocks["low"], 
                               close=stocks["close"], 
                               name='Dow Jones',
                               increasing_line_color= 'green', decreasing_line_color= 'orange'
                         ))
figPx.add_trace(go.Candlestick(x=btc.index, 
                               open=btc["open"], 
                               high=btc["high"], 
                               low=btc["low"], 
                               close=btc["close"],
                               name='Bitcoin',
                               increasing_line_color= 'blue', decreasing_line_color= 'red'
                         ))
figPx.update_layout(hovermode="x unified",
                        plot_bgcolor='rgba(0,0,0,0)',
                        title={'x':0.5,'xanchor': 'center',
                              'font':{'size': 24}
                        },
                        xaxis=dict(
                            title="Year",
                            showgrid=False,  # Removes X-axis grid lines
                            dtick= 'M48'
                        ),
                        yaxis=dict(
                            title="Index Value",  
                            tickformat = ',f',  # Add thousands
                            showgrid=False,  # Removes Y-axis grid lines    
                            zeroline=False
                        ))

# Add source annotation
figPx.update_layout(
    showlegend=False,
    annotations=[
        dict(xref='paper',
            yref='paper',
            x=0.5, y=1.05,
            showarrow=False,
            text ='Source: Retrieved ' + str(end) + ' from Yahoo! Finance.')])

figPx.update_xaxes(showline=True, linewidth=2, linecolor='black',range=['2022-01-01','2022-04-09'])
figPx.update_yaxes(showline=True, linewidth=2, linecolor='black',range=[30000,50000])


## <font color='Red'>Exercício:</font> Análise do desempenho da Dow Jones e IBOVESPA

Passo 1: Carregar dados do índice IBOVESPA do yfinance. 

**Código**: image.png

In [27]:
# Definimos o range para aquisição dos dados
start = dt.datetime(2015, 1, 1)
end = dt.date.today()

ibov = yf.download("^BVSP", start, end)

ibov.reset_index(inplace=True)



ibov.columns = [col.lower() for col in ibov.columns]
ibov['date'] = pd.to_datetime(ibov['date'], yearfirst=True)
ibov.set_index('date', inplace = True)

ibov['date'] = [dt.datetime.strftime(i, "%Y-%m-%d") for i in ibov.index]
ibov['date'] = pd.to_datetime(ibov['date'], yearfirst=True)
ibov.set_index('date', inplace = True)

[*********************100%***********************]  1 of 1 completed


Passo 2: Criando uma visualização de série temporal com os dados da Dow Jones e da IBOVESPA, anotando no gráfico o "começo" da pandemia da COVID-19

In [32]:
figPx = go.Figure()
figPx.add_trace(go.Scatter(x=stocks.index, y=stocks["close"], name='Dow Jones',
                         line=dict(color='blue', width=1)))
figPx.add_trace(go.Scatter(x=ibov.index, y=ibov["close"], name='IBOVESPA',
                         line=dict(color='firebrick', width=1)))
figPx.update_traces(hovertemplate='Value: %{y:.1f}')
figPx.update_layout(hovermode="x unified",
                        plot_bgcolor='rgba(0,0,0,0)',
                        title={'x':0.5,'xanchor': 'center',
                              'font':{'size': 24}
                        },
                        xaxis=dict(
                            title="Year",
                            showgrid=False,  # Removes X-axis grid lines
                            dtick= 'M48'
                        ),
                        yaxis=dict(
                            title="Index Value",  
                            tickformat = ',f',  # Add thousands
                            showgrid=False,  # Removes Y-axis grid lines    
                            zeroline=False
                        ))

# annotate the figure with Covid info
figPx.add_annotation(
            x=dt.datetime(2020, 3, 1),
            y=25000,
            xref="x",
            yref="y",
            text="<b>Covid-19 Declared</b><br><b> a Pandemic</b><br><b> (March 2020)</b>",
            showarrow=True,
            arrowhead=5,
            ax=80,
            ay=-100)

figPx.update_xaxes(showline=True, linewidth=2, linecolor='black')
figPx.update_yaxes(showline=True, linewidth=2, linecolor='black')

<a id="html"></a>
#2. Converter uma Figura em PDF ou Código HTML ([top](#agenda))

### Exportar imagem como imagem estática

Podemos usar plotly para criar imagens legais e com aparência profissional para pdfs. Nesse caso, plotly e matplotlib são rivais. Salvar um arquivo estático como um pdf requer a instalação da dependência `kaleido`:

```
pip install kaleido
```

Depois de concluído, execute `plotly.io.write_image()`. Como apelidei 'plotly.io' como 'pio', meu comando é o seguinte:

In [35]:
!pip install -U kaleido

In [36]:
pio.write_image(figPx, file='fig_Finance.pdf')

ValueError: ignored

Abra seu diretório de trabalho atual e abra o arquivo "fig_Finance.pdf". Alternativamente, podemos salvar nossa figura como um objeto dinâmico em html:

In [37]:
pio.write_html(figPx, file='fig_Finance.html', full_html = False, auto_open=False, config={"displayModeBar": False, "showTips": False, "responsive": True})

No código acima estamos usando o método `write_html` para converter cada figura em um arquivo html.

A operação é a seguinte:

1. Abra um arquivo `html`.

1. Converta a figura (por exemplo, figPx) em código html e grave no arquivo aberto.

1. Feche o arquivo `html`.


Outras coisas:

* `full_html` indica se o arquivo html deve ser escrito como um arquivo independente (ou seja, "True") completo com cabeçalhos ou apenas como um contêiner `<div>` (ou seja, "False").
* `auto_open` informa ao computador se ele deve abrir o arquivo html quando completo. Tente colocar este valor em "True" mas `full_html` em false. O que você acha que acontecerá?
* `config` inclui os comandos para desligar algumas das interações como o menu bvar.

Abra seu diretório de trabalho atual e abra o arquivo "fig_Finance.html" para revelar uma página de trabalho de sua figura. Você pode exportar o código-fonte para sua própria página da Web para compartilhar com outras pessoas.